In [1]:
import pandas as pd
#online link to prostate data: https://web.stanford.edu/~hastie/ElemStatLearn/
path = 'F:/fall-21/ime692/l7/prostate.csv'
df_features = pd.read_csv(path)
names = df_features.columns.values
cols_to_norm = names[1: len(names)-2]
cols_to_norm

array(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason',
       'pgg45'], dtype=object)

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_features[cols_to_norm] = scaler.fit_transform(df_features[cols_to_norm]) #normalize selected variables
#dataS = df_features.iloc[:,1:10]

In [3]:
df_features.head()

,Unnamed: 0,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,1,-1.645861,-2.016634,-1.872101,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-0.430783,T
1,2,-1.999313,-0.725759,-0.791989,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-0.162519,T
2,3,-1.587021,-2.200154,1.368234,-1.030029,-0.525657,-0.867655,0.344407,-0.156155,-0.162519,T
3,4,-2.178174,-0.812191,-0.791989,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-0.162519,T
4,5,-0.510513,-0.461218,-0.251933,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,0.371564,T


In [4]:
#https://stackoverflow.com/questions/35710423/select-a-subset-of-observations-from-a-pandas-dataframe-using-a-list-of-labels
di = df_features.set_index('train')
#di = di.sort_index()
train = di.loc['T']
type(train) # of the object
train.shape #dimensions

(67, 10)

In [5]:
test = di.loc['F']
test.shape #dimension
#len(test.columns.values)

(30, 10)

In [6]:
trainS = train.iloc[:, 1:10]

In [7]:
trainS.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
train,,,,,,,,,
T,-1.645861,-2.016634,-1.872101,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-0.430783
T,-1.999313,-0.725759,-0.791989,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-0.162519
T,-1.587021,-2.200154,1.368234,-1.030029,-0.525657,-0.867655,0.344407,-0.156155,-0.162519
T,-2.178174,-0.812191,-0.791989,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,-0.162519
T,-0.510513,-0.461218,-0.251933,-1.030029,-0.525657,-0.867655,-1.047571,-0.868957,0.371564


In [9]:
X = trainS.iloc[:,0:8] # first eight predictors [0, 8)
y = trainS.iloc[:,[8]]

In [13]:
from sklearn.linear_model import LinearRegression
import numpy as np
reg = LinearRegression().fit(X, y)
np.round(reg.score(X, y), decimals=3)


0.694

In [14]:
np.round(reg.coef_, decimals=3)

array([[ 0.676,  0.262, -0.141,  0.209,  0.304, -0.287, -0.021,  0.266]])

In [11]:
#0.680, 0.262, -0.141, 0.210, 0.304, -0.287, -0.21, 0.266

In [15]:
np.round(reg.intercept_, decimals=3)

array([2.465])

In [16]:
testS = test.iloc[:, 1:10]
Xt = testS.iloc[:, 0:8]
yt = testS.iloc[:,[8]]
yhat = reg.predict(Xt)

In [17]:
def mse(x):
    errSq = x.apply(lambda x: x**2)
    return errSq["lpsa"].mean()


In [18]:
#errSq = err.apply(lambda x: x**2)
#errSq["lpsa"].mean()
err = yt - yhat
np.round(mse(err), decimals=3)

0.521

#ridge regression
#numpy arange
    

In [19]:
#ridge regression 
#numpy arange for grid search
import numpy as np
z  = np.linspace(-2, 4, num=101)

In [20]:
b = 10*np.ones(len(z))
lm = np.power(b, z)

In [22]:
#10fold cv to pick the best lambda
from sklearn.linear_model import RidgeCV
clf = RidgeCV(alphas = lm, cv=10) #10-fold cross validation
clf.fit(X, y)
lmd = clf.alpha_ # 45.71
np.round(lmd, decimals=3)

45.709

In [23]:
#ridge regression corresponding to best lamda
from sklearn.linear_model import Ridge
rdm = Ridge(alpha = lmd)
rdm.fit(X, y)

Ridge(alpha=45.708818961487474)

In [24]:
np.round(rdm.coef_, decimals=3) #ols: 0.680, 0.262, -0.141, 0.210, 0.304, -0.287, -0.21, 0.266

array([[ 0.338,  0.213, -0.016,  0.136,  0.196,  0.051,  0.048,  0.113]])

In [25]:
#prediction using the fitted model
yrt = rdm.predict(Xt) #predict using the ridge model on test data

In [26]:
np.round(mse(yrt-yt), decimals=3) #mse for ridge

0.508

In [27]:
np.round(rdm.intercept_, decimals=3)

array([2.46])